In [50]:
import pandas as pd
import numpy as np
df = pd.read_csv('../../results/result.csv')

repo_num = 6990
# merge_tool = "gitmerge-recursive-patience"
# merge_tool = "gitmerge-recursive-minimal"
# merge_tool = "spork"
# merge_tool = "gitmerge-resolve"
# merge_tool = "intellimerge"
# merge_tool = "gitmerge-ort-ignorespace"
merge_tool = "gitmerge-ort"

script = "../scripts/merge_tools/" + merge_tool + ".sh"

In [51]:
from merge_tester import MERGE_STATE
df[merge_tool].apply(lambda x: MERGE_STATE[x])[0]

<MERGE_STATE.Tests_passed: 5>

In [52]:
from validate_repos import clone_repo
repo = clone_repo(df.iloc[repo_num]["repo_name"]) # Return a Git-Python repo object

uniVocity/univocity-parsers  : Cloning repo


Repository 'uniVocity/univocity-parsers' cloned to path: ./repos/uniVocity/univocity-parsers
uniVocity/univocity-parsers  : Finished cloning


In [53]:
# For a git to work you need to pass a name for each branch
repo.remote().fetch()
repo.git.checkout(df.iloc[repo_num]["left"], force=True)
repo.submodule_update()
repo.git.checkout("-b", "TEMP_LEFT_BRANCH", force=True)
repo.git.checkout(df.iloc[repo_num]["right"], force=True)
repo.submodule_update()
repo.git.checkout("-b", "TEMP_RIGHT_BRANCH", force=True)

''

In [54]:
repo_path = repo.git.rev_parse("--show-toplevel")
repo_path

'/home/benrr/miniconda3/envs/merge/AST-Merging-Ben-Analysis/src/python/repos/uniVocity/univocity-parsers'

In [55]:
import subprocess
# Returns = 1 process fails
subprocess.call([script,repo_path,"TEMP_LEFT_BRANCH","TEMP_RIGHT_BRANCH"])

Path: /home/benrr/miniconda3/envs/merge/AST-Merging-Ben-Analysis/src/python/repos/uniVocity/univocity-parsers
Running: git merge --no-edit -s ort TEMP_RIGHT_BRANCH
Auto-merging README.md
Auto-merging pom.xml
CONFLICT (content): Merge conflict in pom.xml
Auto-merging src/test/java/com/univocity/parsers/issues/github/Github_31.java
CONFLICT (content): Merge conflict in src/test/java/com/univocity/parsers/issues/github/Github_31.java
Automatic merge failed; fix conflicts and then commit the result.


Switched to branch 'TEMP_LEFT_BRANCH'


1

In [56]:
subprocess.call(["cd",repo_path,"&&","git","merge","--abort"])

FileNotFoundError: [Errno 2] No such file or directory: 'cd'

In [ ]:
# Merge Sucess (Return 0)
subprocess.call(["../scripts/merge_tools/gitmerge-ort-ignorespace.sh",repo_path,"NAME_OF_LEFT_BRANCH","NAME_OF_RIGHT_BRANCH"])
# Be careful the merge has been completed the history of the left branch has now changed and you can't 
# merge again unless you delete the repo and clone it again